## MinHashing Articles

In [1]:
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

conf = SparkConf().setMaster("local[*]").setAppName("MinHashing Articles")
sc = SparkContext(conf = conf)
spark = SparkSession(sc)

### Articles table

In [2]:
articles = spark.read.load("../output/test/articles")

articles.toPandas().head()

,abstract,body,categories,title,id,words,filtered,tokens,rawFeatures,features
0,A e r o s o l g e n e r a t i o n a n d ...,"In 2010, the Basic Fitness Test (BFT) became c...",[Research],Correlation between the results of three physi...,85899345920,"[in, 2010, the, basic, fitness, test, bft, bec...","[2010, basic, fitness, test, bft, became, comp...",56,"(3.0, 2.0, 1.0, 1.0, 2.0, 4.0, 1.0, 4.0, 4.0, ...","(0.6107968657237186, 0.5637023042819753, 0.281..."
1,Müller glial cells are important regulators o...,"Due to the substantive time, cost, and animal ...",[Review],Aerosol generation and characterization of mul...,85899345921,"[due, to, the, substantive, time, cost, and, a...","[due, substantive, time, cost, animal, numbers...",385,"(13.0, 16.0, 24.0, 22.0, 12.0, 11.0, 13.0, 14....","(2.6467864181361143, 4.5096184342558026, 6.764..."
2,Approximately 40–60 % of patients remain suff...,The role of retinal cells itself in sight-thre...,[Research],"Aquaporin 11, a regulator of water efflux at r...",85899345922,"[the, role, of, retinal, cells, itself, in, si...","[role, retinal, cells, sight, threatening, com...",49,"(1.0, 4.0, 3.0, 6.0, 2.0, 2.0, 4.0, 2.0, 4.0, ...","(0.20359895524123955, 1.1274046085639506, 0.84..."
3,M e a n a b s o l u t e c h a n g e o f ...,"With 170,000 reported TB cases in the year 201...",[Research Article],Lung functions among patients with pulmonary t...,85899345923,"[with, 170, 000, reported, tb, cases, in, the,...","[170, 000, reported, tb, cases, year, 2014, ta...",18,"(0.0, 0.0, 2.0, 2.0, 0.0, 0.0, 0.0, 2.0, 2.0, ...","(0.0, 0.0, 0.5637023042819753, 0.4951241573247..."
4,Juvenile idiopathic arthritis (JIA) is an aut...,"Increasing complexity of both, the therapy (p...",[Research Article],Impact of a community pharmacist-led medicatio...,85899345924,"[increasing, complexity, of, both, the, therap...","[increasing, complexity, therapy, polypharmacy...",28,"(1.0, 0.0, 1.0, 2.0, 2.0, 1.0, 1.0, 3.0, 0.0, ...","(0.20359895524123955, 0.0, 0.28185115214098766..."


### Articles partitioned by hashes

In [3]:
buckets = spark.read.load("../output/test/articles_partitioned_by_hashes")

buckets.toPandas()

,hashes,ids,titles
0,"[[111046356.0], [989858842.0]]","[8589934594, 111669149701, 120259084291, 12884...",[An epidemiological study of rates of illness ...
1,"[[111046356.0], [989993821.0]]","[51539607555, 94489280514, 77309411331, 146028...",[Bitpacking techniques for indexing genomes: I...
2,"[[154535185.0], [990398758.0]]",[111669149697],[Efficacy of Intra-aortic Balloon Pump before ...
3,"[[154535185.0], [989858842.0]]","[0, 17179869188, 146028888067, 77309411330, 34...",[The Lung Screen Uptake Trial (LSUT): protocol...
4,"[[154535185.0], [990533737.0]]",[128849018882],[Possible Mechanism of Therapeutic Effect of 3...
5,"[[241512843.0], [989858842.0]]",[163208757251],[Validation of a symphysis-fundal height chart...
6,"[[804900880.0], [989858842.0]]","[111669149698, 120259084292]",[MiR-106b-5p Inhibits Tumor Necrosis Factor-α-...
7,"[[198024014.0], [989858842.0]]","[154618822661, 34359738372, 51539607556]","[Mood, anxiety, and alcohol use disorders and ..."
8,"[[111046356.0], [990263779.0]]","[128849018885, 111669149699]",[Association between Carotid Intima-media Thic...
9,"[[111046356.0], [990128800.0]]","[94489280512, 128849018883, 94489280513]",[Regeneration of the lung: Lung stem cells and...


### Jaccard distances of article pairs

In [4]:
jaccardDistances = spark.read.load("../output/test/articles_jaccard_distances")

jaccardDistances.toPandas()

,id1,id2,jaccardDistance
0,154618822658,94489280515,0.350000
1,163208757250,77309411328,0.350000
2,42949672960,103079215105,0.350000
3,137438953475,163208757250,0.350000
4,137438953477,154618822661,0.350000
...,...,...,...
9165,154618822660,60129542144,0.125000
9166,103079215104,120259084289,0.214286
9167,120259084289,103079215104,0.214286
9168,128849018885,137438953473,0.454545


In [5]:
sc.stop()